In [1]:
import numpy as np
import pandas as pd
import os
import time
import pandas_gbq
from google.cloud import bigquery
import re


In [2]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Dados para subir\\Tabelas finais')


In [3]:
df = pd.read_csv('SIAPE_remuneracao_liquida_executivo_raca_escolaridade.csv')
df

,ano,escolaridade,raca,liquido
0,1999,1º grau completo ou menos,Amarela,"3320,7"
1,1999,1º grau completo ou menos,Branca,"3543,9"
2,1999,1º grau completo ou menos,Indígena,"3015,5"
3,1999,1º grau completo ou menos,Não informado,"3254,2"
4,1999,1º grau completo ou menos,Negra,"3312,3"
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,"9471,4"
326,2020,Superior completo ou mais,Branca,"9089,9"
327,2020,Superior completo ou mais,Indígena,"7412,6"
328,2020,Superior completo ou mais,Não informado,"9111,1"


In [4]:
df.rename(columns={'escolaridade':'grau_instrucao','raca':'cor_raca', 'liquido':'remuneracao_liquida'}, inplace=True) #padronizando
df


,ano,grau_instrucao,cor_raca,remuneracao_liquida
0,1999,1º grau completo ou menos,Amarela,"3320,7"
1,1999,1º grau completo ou menos,Branca,"3543,9"
2,1999,1º grau completo ou menos,Indígena,"3015,5"
3,1999,1º grau completo ou menos,Não informado,"3254,2"
4,1999,1º grau completo ou menos,Negra,"3312,3"
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,"9471,4"
326,2020,Superior completo ou mais,Branca,"9089,9"
327,2020,Superior completo ou mais,Indígena,"7412,6"
328,2020,Superior completo ou mais,Não informado,"9111,1"


In [5]:
df['remuneracao_liquida'] = df['remuneracao_liquida'].str.replace(',','.').astype(float)
df

,ano,grau_instrucao,cor_raca,remuneracao_liquida
0,1999,1º grau completo ou menos,Amarela,3320.7
1,1999,1º grau completo ou menos,Branca,3543.9
2,1999,1º grau completo ou menos,Indígena,3015.5
3,1999,1º grau completo ou menos,Não informado,3254.2
4,1999,1º grau completo ou menos,Negra,3312.3
...,...,...,...,...
325,2020,Superior completo ou mais,Amarela,9471.4
326,2020,Superior completo ou mais,Branca,9089.9
327,2020,Superior completo ou mais,Indígena,7412.6
328,2020,Superior completo ou mais,Não informado,9111.1


In [6]:
df['grau_instrucao'].unique()

array(['1º grau completo ou menos', '2º grau completo',
       'Superior completo ou mais'], dtype=object)

In [7]:
df['cor_raca'].unique()

array(['Amarela', 'Branca', 'Indígena', 'Não informado', 'Negra'],
      dtype=object)

In [8]:
def transforme(nome):
    nome =re.sub(r"\b1º grau completo ou menos\b", "Até Ensino Fundamental", nome)
    nome =re.sub(r"\b2º grau completo\b", "Até Ensino Médio", nome)
    nome =re.sub(r"\bSuperior completo ou mais\b", "Até superior completo ou mais", nome)
    return nome

In [9]:
df['grau_instrucao'] = df['grau_instrucao'].apply(transforme)

In [10]:
df['grau_instrucao'].unique()

array(['Até Ensino Fundamental', 'Até Ensino Médio',
       'Até superior completo ou mais'], dtype=object)

In [11]:
df['raca_escolaridade'] = df['grau_instrucao'] + ' / ' + df['cor_raca']


In [12]:
df = df[['ano', 'grau_instrucao', 'cor_raca', 'raca_escolaridade', 'remuneracao_liquida' ]]


In [13]:
df

,ano,grau_instrucao,cor_raca,raca_escolaridade,remuneracao_liquida
0,1999,Até Ensino Fundamental,Amarela,Até Ensino Fundamental / Amarela,3320.7
1,1999,Até Ensino Fundamental,Branca,Até Ensino Fundamental / Branca,3543.9
2,1999,Até Ensino Fundamental,Indígena,Até Ensino Fundamental / Indígena,3015.5
3,1999,Até Ensino Fundamental,Não informado,Até Ensino Fundamental / Não informado,3254.2
4,1999,Até Ensino Fundamental,Negra,Até Ensino Fundamental / Negra,3312.3
...,...,...,...,...,...
325,2020,Até superior completo ou mais,Amarela,Até superior completo ou mais / Amarela,9471.4
326,2020,Até superior completo ou mais,Branca,Até superior completo ou mais / Branca,9089.9
327,2020,Até superior completo ou mais,Indígena,Até superior completo ou mais / Indígena,7412.6
328,2020,Até superior completo ou mais,Não informado,Até superior completo ou mais / Não informado,9111.1


In [14]:
df['raca_escolaridade'].unique()

array(['Até Ensino Fundamental / Amarela',
       'Até Ensino Fundamental / Branca',
       'Até Ensino Fundamental / Indígena',
       'Até Ensino Fundamental / Não informado',
       'Até Ensino Fundamental / Negra', 'Até Ensino Médio / Amarela',
       'Até Ensino Médio / Branca', 'Até Ensino Médio / Indígena',
       'Até Ensino Médio / Não informado', 'Até Ensino Médio / Negra',
       'Até superior completo ou mais / Amarela',
       'Até superior completo ou mais / Branca',
       'Até superior completo ou mais / Indígena',
       'Até superior completo ou mais / Não informado',
       'Até superior completo ou mais / Negra'], dtype=object)

In [18]:
df['cor_raca'].unique()

array(['Amarelas', 'Brancas', 'Indígenas', 'sem informação de raça',
       'Negras'], dtype=object)

In [16]:
def transforme(nome):
    nome =re.sub(r"\bAmarela\b", "Amarelas", nome)
    nome =re.sub(r"\bBranca\b", "Brancas", nome)
    nome =re.sub(r"\bIndígena\b", "Indígenas", nome)
    nome =re.sub(r"\bNão informado\b", "Sem informação de raça", nome)
    nome =re.sub(r"\bNegra\b", "Negras", nome)

    return nome

In [17]:
df['cor_raca'] = df['cor_raca'].apply(transforme)

Subindo para o GBQ

In [19]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [20]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'), 
 bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('raca_escolaridade','STRING',description='Raça/cor e escolaridade da pessoa observada'),
 bigquery.SchemaField('remuneracao_liquida','FLOAT',description='Remuneração liquida da pessoa observada')]

In [21]:
table_ref = dataset_ref.table('SIAPE_remuneracao_liquida_executivo_raca_escolaridade')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=843272f7-65d5-43cd-b224-5307d5cac8d6>